In [ ]:
import mne
import numpy as np

import copy
import pickle
import scipy

import os
os.chdir('..')
from _parameters import *

from IPython.display import clear_output

In [ ]:
def open_tfr(s, moment, modal):
    
    # Dict per cond
    tfr = {c: None for c in cond[modal]}

    # Read tfr per cond
    for c in cond[modal]:

        tfr_fname = dirs['tfr'] + '/tfr_multi_' + moment + '_' + c.replace('/', '_') + '_s' + str(s) + '.h5'    

        # Add to dict
        tfr[c] = mne.time_frequency.read_tfrs(fname=tfr_fname)[0]

    return tfr

In [ ]:
def calc_cvsi(tfr, trials_L, trials_R, chan_L, chan_R):

    tfr_cpy = copy.deepcopy(tfr)

    # Left channels
    contra_L = tfr_cpy[trials_R].pick(chan_L).data.mean(0) # contra
    ipsi_L = tfr_cpy[trials_L].pick(chan_L).data.mean(0) # ipsi

    cvsi_L = ((contra_L - ipsi_L) / (contra_L + ipsi_L)) * 100

    tfr_cpy = copy.deepcopy(tfr)
    
    # Right channels
    contra_R = tfr_cpy[trials_L].pick(chan_R).data.mean(0) # contra
    ipsi_R = tfr_cpy[trials_R].pick(chan_R).data.mean(0) # ipsi

    cvsi_R = ((contra_R - ipsi_R) / (contra_R + ipsi_R)) * 100

    # Average
    cvsi = np.mean(np.asarray([cvsi_L, cvsi_R]), 0)

    return cvsi

In [ ]:
def get_cvsi(subs, moment):

    # Empty list for each cond
    cvsi_cond = ['vis', 'mot']
    cvsi_dat = {c:[] for c in cvsi_cond}

    # Loop over subs
    for s in subs:

        # Get tfr data
        tfr_vis = open_tfr(s, moment, 'visual')
        tfr_mot = open_tfr(s, moment, 'motor')

        # vis and mot
        cvsi_dat['vis'].append(
            calc_cvsi(tfr_vis, 'itemL', 'itemR', 'PO7', 'PO8')
        )
        cvsi_dat['mot'].append(
            calc_cvsi(tfr_mot, 'respL', 'respR', 'C3', 'C4')
        )

        clear_output(wait = False)

    # Avg over subs
    cvsi_avg = {c:None for c in cvsi_cond}

    for c in cvsi_dat.keys():
        cvsi_avg[c] = np.mean(np.asarray(cvsi_dat[c]),0)

    return cvsi_dat, cvsi_avg

In [ ]:
def get_cvsi_samerev(subs, moment):

    # Empty list for each cond
    cvsi_cond = ['vis/same', 'vis/rvrs', 'mot/same', 'mot/rvrs']
    cvsi_dat = {c:[] for c in cvsi_cond}

    # Loop over subs
    for s in subs:

        # Get tfr data
        tfr_vis_samerev = open_tfr(s, moment, 'vis-samerev')
        tfr_mot_samerev = open_tfr(s, moment, 'mot-samerev')

        # calc cvsi per cond
        cvsi_dat['vis/same'].append(
            calc_cvsi(tfr_vis_samerev, 'same/itemL', 'same/itemR', 'PO7', 'PO8')
        )
        cvsi_dat['vis/rvrs'].append(
            calc_cvsi(tfr_vis_samerev, 'rvrs/itemL', 'rvrs/itemR', 'PO7', 'PO8')
        )
        cvsi_dat['mot/same'].append(
            calc_cvsi(tfr_mot_samerev, 'same/respL', 'same/respR', 'C3', 'C4')
        )
        cvsi_dat['mot/rvrs'].append(
            calc_cvsi(tfr_mot_samerev, 'rvrs/respL', 'rvrs/respR', 'C3', 'C4')
        )

        clear_output(wait = False)

    # Avg over subs
    cvsi_avg = {c:None for c in cvsi_cond}

    for c in cvsi_dat.keys():
        cvsi_avg[c] = np.mean(np.asarray(cvsi_dat[c]),0)

    return cvsi_dat, cvsi_avg

In [ ]:
def get_tc(data, fband: list, frange = [3,40]):

    # Freq band index
    flow, fhigh = [f-frange[0] for f in fband]

    # Get freq avg time-course
    tc = np.asarray(data)

    tc = tc[:, flow:fhigh+1, :]
    tc = np.mean(tc, 1)

    # Gaussian filter
    tc = scipy.ndimage.gaussian_filter1d(tc,10)

    return tc

In [ ]:
def tfr_to_tc(moment):
    
    # Load data
    cvsi_avg_f = open(dirs['cvsi'] + '/cvsi_' + moment + '.pkl', 'rb')
    cvsi_dat, _ = pickle.load(cvsi_avg_f)

    cvsi_avg_f.close()

    # Open a TFR file
    tfr_fname = dirs['tfr'] + '/tfr_multi_' + moment + '_itemL_s1.h5'
    tfr = mne.time_frequency.read_tfrs(fname=tfr_fname)[0]

    # Get time-course data for all participants

    cvsi_tc = {c:None for c in cvsi_dat.keys()}

    for c in cvsi_dat.keys():

        if 'vis' in c: fband = [8,12]
        elif 'mot' in c: fband = [13,30]

        tc = get_tc(cvsi_dat[c], fband)

        cvsi_tc[c] = tc

    cvsi_tc['time'] = tfr.times

    # Save time-courses

    cvsi_tc_f = open(dirs['cvsi'] + '/cvsi_tc_' + moment + '.pkl', 'wb')

    pickle.dump(cvsi_tc, cvsi_tc_f)
    cvsi_tc_f.close()

In [ ]:
def run_cvsi(moment, samerev=False):
    if samerev:
        cvsi_dat, cvsi_avg = get_cvsi_samerev(subjects, moment)
    else:
        cvsi_dat, cvsi_avg = get_cvsi(subjects, moment)

    # Save as pickle
    cvsi_dat_f = open(dirs['cvsi'] + '/cvsi_' + moment + '.pkl', 'wb')
    pickle.dump([cvsi_dat, cvsi_avg], cvsi_dat_f)
    cvsi_dat_f.close()

    tfr_to_tc(moment)

In [ ]:
# Locked to encoding

run_cvsi('enc1')

In [ ]:
# Locked to probe (1)

run_cvsi('prob1')

In [ ]:
# Locked to probe (2)

run_cvsi('prob2')

In [ ]:
# Locked to resp (1)

run_cvsi('resp1')


In [ ]:
# Locked to resp (2)

run_cvsi('resp2')